In [1]:
# execute to import notebook styling for tables and width etc.
from IPython.core.display import HTML
import urllib.request
response = urllib.request.urlopen('https://raw.githubusercontent.com/DataScienceUWL/DS775v2/master/ds755.css')
HTML(response.read().decode("utf-8"));


# **HW3.1** - Transportation with Capacity Constraint

In addition to the supply and demand constraints sometimes transportation problems can have capacity restrictions such that each route will have a maximum number of units that may be shipped along that route. (Similar to worker availability in the worker scheduling problem.)

For this problem there are 3 warehouses that ship the same product to 21 stores.  The supply at the warehouses, demand at the stores, unit shipping costs, and route capacities are stored in the `transp_prob1.xlsx` file in the `data` folder.  Use the `openpyxl` package to read the problem data into lists and nested lists.  Once you have the problem data converted into dictionaries, then adapt the transportation code in the lesson to minimize the total shipping cost.  You'll need to add the extra capacity constraint to give an upper bound to the number of units shipped on each route.  Your code should also use Technique 3 to eliminate unneeded decision variables.

To get you started the code cell below shows how to use `openpyxl` to read ranges from the Excel workbook.  For full credit you must use an abstract approach to the solution code and you should display the output in a nicely formatted manner (you could use a pandas data frame to display the transported amounts).

<font color = "blue"> *** 10 points -  finish the code in the cell below *** (don't delete this cell) </font>

In [2]:
import pandas as pd
from pyomo.environ import *
from dataclasses import make_dataclass

# using openpyxl
from openpyxl import load_workbook
wb = load_workbook(filename='./data/transp_prob_1.xlsx', data_only=True)
sheet = wb.active


# specify upper left and lower right cells, returns a list or list of lists representing rows
def read_range(sheet, begin, end):
    table = sheet[begin:end]
    height = len(table)
    width = len(table[0])
    if height == 1 or width == 1:
        # for a single row or column produce a list
        tmp = [cell.value for row in table for cell in row]
    else:
        # for an array of cells produces a list of row lists
        tmp = [[cell.value for cell in row] for row in table]
    return (tmp)


# finish reading the data
warehouses = read_range(sheet, 'A3', 'A5')
stores = read_range(sheet, 'B3', 'B22')
wares_stores = [(w,s) for [w,s] in read_range(sheet,'D3','E31')]
capacity_dict = {(w,s):cap for [w,s,cap] in read_range(sheet,'D3','F31')}
cost_dict = {(w,s):cost for [w,s,cap,cost] in read_range(sheet,'D3','G31')}
supply_dict = { w:q for [w,q] in read_range(sheet,'I3','J5')}
demand_dict = { s:d for [s,d] in read_range(sheet,'L3','M22')}


# throw an error if total supply and demand do not match
assert (sum(supply_dict.values()) == sum(demand_dict.values()))


# build the model
model = ConcreteModel()

model.transp = Var(wares_stores, domain=NonNegativeReals)

model.total_cost = Objective(expr=sum(cost_dict[(w, s)] * model.transp[w, s] for w,s in wares_stores), sense=minimize)

# constraints
model.supply_ct = ConstraintList()
for w in warehouses:
    model.supply_ct.add(sum(model.transp[w, s] for s in stores if (w,s) in wares_stores) == supply_dict[w])

model.demand_ct = ConstraintList()
for s in demand_dict.keys():
    model.demand_ct.add(sum(model.transp[w, s] for w in warehouses if (w, s) in wares_stores) == demand_dict[s])
    
model.capacity = ConstraintList()
for (w,s) in wares_stores:
    model.capacity.add(model.transp[w, s] <= capacity_dict[(w, s)])

# solve
solver = SolverFactory('glpk')
solver.solve(model)

# display
print(f"Shipping Costs = ${model.total_cost():,.2f}\n")
print("Recommended Units by Route:\n")
Route = make_dataclass("Route", [("Warehouse", str), ("Store", str), ("Units", float)])
output = [Route(w, s, model.transp[w, s]()) for (w, s) in wares_stores]
df = pd.DataFrame(output)
df.pivot_table(index="Store", columns="Warehouse", values="Units", margins=True, aggfunc='sum', margins_name="Totals").fillna("")

# The table looks better with Stores as columns and Warehouses as rows, but some columns aren't displayed in Jupyter when the table is that wide...   

Shipping Costs = $29,827.00

Recommended Units by Route:



Warehouse,wA,wB,wC,Totals
Store,,,,
sA,173.0,,,173.0
sB,103.0,,,103.0
sC,110.0,,,110.0
sD,121.0,,,121.0
sE,148.0,,,148.0
sF,145.0,,,145.0
sG,100.0,54.0,,154.0
sH,23.0,100.0,,123.0
sI,100.0,65.0,,165.0


# **HW3.2** Shipping Wood to Market

This problem is Case 9.1 on page 370 of the textbook.

Alabama Atlantic is a lumber company that has three sources of wood and five markets to be supplied. The annual availability of wood at sources 1, 2, and 3 is 15, 20, and 15 million board feet, respectively. The amount that can be sold annually at markets 1, 2, 3, 4, and 5 is 11, 12, 9, 10, and 8 million board feet, respectively.  In the past the company has shipped the wood by train. However, because shipping costs have been increasing, the alternative of using ships to make some of the deliveries is being investigated. This alternative would require the company to invest in some ships. Except for these investment costs, the shipping costs in thousands of dollars per million board feet by rail and by water (when feasible) would be the following for each route:

<img src="images/wood_table_1.png" width="600">

The capital investment (in thousands of dollars) in ships required for each million board feet to be transported annually by ship along each route is given as follows:

<img src = "images/wood_table_2.png" width="600">

Considering the expected useful life of the ships and the time value of money, the equivalent uniform annual cost of these investments is one-tenth the amount given in the table. The objective is to determine the overall shipping plan that minimizes the total equivalent uniform annual cost (including shipping costs).

You are the head of the team that has been assigned the task of determining this shipping plan for each of the following three options.
* *Option 1:* Continue shipping exclusively by rail.
* *Option 2:* Switch to shipping exclusively by water (except where only rail is feasible).
* *Option 3:* Ship by either rail or water, depending on which is less expensive for the particular route.

Present your results for each option. Compare.  You should write a short summary of the results and include a recommendation for the best option.  Be sure to communicate enough that management can implement your solution including which shipping method to use when applicable. You could modularize your code, though this isn't required, by writing a function that takes the cost array as input and outputs the total cost and transportation amounts.  You could then call the function three times for each of the three options.  For full credit you must use an abstract approach to the solution code.

<font color = "blue"> *** 15 points -  answer in cells below *** (don't delete this cell) </font>

In [3]:
from pyomo.environ import *
import pandas as pd

# data parameters
sources = [1, 2, 3]
markets = [1, 2, 3, 4, 5]
wood_availability = [15, 20, 15]
market_amts = [11, 12, 9, 10, 8]
# where ship isn't feasible
bigM = 1000000

ship_invest = [[275, 303, 238, bigM, 285], [293, 318, 270, 250, 265], [bigM, 283, 275, 268, 240]]

wood_avail_dict = dict(zip(sources, wood_availability))
market_amt_dict = dict(zip(markets, market_amts))

# cost arrays
rail_cost = [[61, 72, 45, 55, 66],[69, 78, 60, 49, 56],[59, 66, 63, 61, 47]]
ship_cost = [[31, 38, 24, bigM, 35], [36, 43, 28, 24, 31], [bigM, 33, 36, 32, 26]]
ship_invest_to_add = [[(.1 * amt) for amt in amts] for amts in ship_invest]
ship_cost_final = [[],[],[]]
for i in range(0,len(ship_invest)):
    for j in range(0,(len(ship_invest[i]))):
        to_add = ship_invest_to_add[i][j]+ship_cost[i][j]
        if to_add > bigM:
            ship_cost_final[i].append(rail_cost[i][j])
        else:
            ship_cost_final[i].append(ship_invest_to_add[i][j]+ship_cost[i][j])
cheapest_option = [[],[],[]]
cheapest_method = [[],[],[]]
for i in range(0,len(rail_cost)):
    for j in range(0,len(rail_cost[i])):
        if ship_cost[i][j] == bigM:
            cheapest_option[i].append(rail_cost[i][j])
            cheapest_method[i].append("Rail")
        elif rail_cost[i][j] < ship_cost_final[i][j]:
            cheapest_option[i].append(rail_cost[i][j])
            cheapest_method[i].append("Rail")
        elif rail_cost[i][j] == ship_cost_final[i][j]:
            cheapest_option[i].append(ship_cost_final[i][j])
            cheapest_method[i].append("Either")
        else:
            cheapest_option[i].append(ship_cost_final[i][j])
            cheapest_method[i].append("Ship")
method_df = pd.DataFrame(cheapest_method)
method_df.index=range(1,len(sources)+1)
method_df.columns=range(1,len(markets)+1)
# model function
def shippingModel(cost_array, method):
    
    temp_df = pd.DataFrame(cost_array, columns=markets, index=sources)
    ship_cost_dict = {(s, m): temp_df.loc[s, m] for s in sources for m in markets}
    possible_routes = list(ship_cost_dict.keys())

    # build model
    model = ConcreteModel()
    
    # define variables
    model.wood = Var(possible_routes, domain=NonNegativeReals)

    # define objective function
    model.total_cost = Objective(expr=sum(ship_cost_dict[s, m] * model.wood[s, m] for (s, m) in possible_routes), sense=minimize)

    # define constraints
    model.supply_ct = ConstraintList()
    for s in sources:
        model.supply_ct.add( sum( model.wood[s,m] for m in markets if (s, m) in possible_routes ) == wood_avail_dict[s] )

    model.demand_ct = ConstraintList()
    for m in markets:
        model.demand_ct.add( sum( model.wood[s,m] for s in sources if (s, m) in possible_routes) == market_amt_dict[m] ) 

    # solve
    solver = SolverFactory('glpk')
    solver.solve(model)
    
    # display
    print(f"Total Shipping Cost ({method}) = ${model.total_cost() * 1000:,.2f}\n")
    print("Transported Amounts:\n")
    wood = pd.DataFrame(0, columns=markets, index=sources)
    for (s, m) in possible_routes:
        wood.loc[s, m] = model.wood[s, m].value
    print(wood)
    print("\n---------------------------------------------------\n")

shippingModel(rail_cost, "Rail")
shippingModel(ship_cost_final, "Ship")
shippingModel(cheapest_option, "Lowest Cost Option")
print("\nRecommended Form of Transport:")
print(method_df)

Total Shipping Cost (Rail) = $2,816,000.00

Transported Amounts:

   1   2  3   4  5
1  6   0  9   0  0
2  2   0  0  10  8
3  3  12  0   0  0

---------------------------------------------------

Total Shipping Cost (Ship) = $2,770,800.00

Transported Amounts:

   1   2  3   4  5
1  6   0  9   0  0
2  5   0  0  10  5
3  0  12  0   0  3

---------------------------------------------------

Total Shipping Cost (Lowest Cost Option) = $2,729,100.00

Transported Amounts:

   1   2  3   4  5
1  6   0  9   0  0
2  5   0  0  10  5
3  0  12  0   0  3

---------------------------------------------------


Recommended Form of Transport:
      1     2     3       4     5
1  Ship  Ship  Rail    Rail  Ship
2  Ship  Ship  Ship  Either  Rail
3  Rail  Ship  Rail    Ship  Rail


<font color="green">
Using rail as the only transport method is the most expensive option.  Shipping by sea (unless only rail is feasible) costs less.  Choosing the cheapest method for each route yields the lowest total shipping cost. For each source:market combination, management should choose the cheaper of rail or shipping by sea using the table avove. It will yield $86,900 in savings.
</font>

# **HW3.3** - A more complex transportation problem

Five different products pA,..., pE, must be transported from factories to warehouses and then from the warehouses to stores.  The cost per unit varies along different route segments.  Your job is to minimize the total transportation cost.  The constraints are as follows:
* the amount of each product at each factory must match the supply amount
* the amount of each product delivered to each store must match the demand amount
* the amount of each product shipped to each warehouse must be the same as the amount of each product shipped from each warehouse
* the total (summed) amount of all products at each warehouse must be $\leq$ `MaxStorage`
* the total (summed) amount of all products shipped from each factory to each warehouse must be $\leq$ `CapacityFW`
* the total (summed) amount of all products shipped from each warehouse to each store must be $\leq$ `CapacityWS`

The data for this problem is in `transp_prob3.xlsx`.  Since many routes aren't used you'll need to use one of the techniques in the lesson to make sure that no product is shipped along an unused route. 

Your solution should display the total transportation cost as well as the amount transported from both factories to warehouses and from warehouses to stores.  The transport multiple products example in the lesson is a good place to start.  Note that you'll need two "arrays" of decision variables for the two sets of connections. You'll also have two separate sums in your objective function to compute the tranportation cost. For full credit you must use an abstract approach to the solution code.

There are a couple of different ways to approach this problem:
1.  Add a constraint so that the total amount of each product shipped into the each warehouse is the same as the total amount shipped out of the warehouse.
2.  Introduce some extra decision variables for the amount of each product at each warehouse.  These variables are then the demand amounts for the shipping from factories to warehouses and the supply amounts for the shipping from warehouses to factories.  With the extra variables the two F -> W and W -> S problems are pretty much standard supply and demand problems.

Two things to note:
1.  You'll need separate sets of decision variables for the number of each product from factories to warehouses and for the number of each product from warehouses to stores.
2.  You can manually check (some of) your solution output against the constraints to make sure they're working the way you intend. 

<font color = "blue"> *** 15 points -  answer in cell below *** (don't delete this cell) </font>

In [214]:
import pandas as pd
from pyomo.environ import *
from dataclasses import make_dataclass

# using openpyxl
from openpyxl import load_workbook
wb = load_workbook(filename='./data/transp_prob3.xlsx', data_only=True)
sheet = wb.active


# specify upper left and lower right cells, returns a list or list of lists representing rows
def read_range(sheet, begin, end):
    table = sheet[begin:end]
    height = len(table)
    width = len(table[0])
    if height == 1 or width == 1:
        # for a single row or column produce a list
        tmp = [cell.value for row in table for cell in row]
    else:
        # for an array of cells produces a list of row lists
        tmp = [[cell.value for cell in row] for row in table]
    return (tmp)


# data
factories = read_range(sheet, 'A2', 'A6')
warehouses = read_range(sheet, 'B2', 'B11')
stores = read_range(sheet, 'C2', 'C21')
products = read_range(sheet, 'D2', 'D6')
capacityFW = read_range(sheet, 'E2', 'E2')
capacityWS = read_range(sheet, 'E5', 'E5')
maxStorage = read_range(sheet, 'E8', 'E8')
FWcost = {(p, f, w): c for [p, f, w, c] in read_range(sheet, 'G3', 'J71')}
FWroutes = {(p, f, w) for [p, f, w] in read_range(sheet, 'G3', 'I71')}
WScost = {(p, w, s): c for [p, w, s, c] in read_range(sheet, 'L3', 'O202')}
WSroutes = {(p, w, s) for [p, w, s] in read_range(sheet, 'L3', 'N202')}
supply = {(p, f): q for [p, f, q] in read_range(sheet, 'Q3', 'S15')}
demand = {(p, s): q for [p, s, q] in read_range(sheet, 'U3', 'W102')}
wares_stores = {(w, s) for (p, w, s) in WSroutes}
fact_wares = {(f, w) for (p, f, w) in FWroutes}

# throw an error if total supply and demand do not match
for p in products:
    assert (sum(supply[p, f] for f in factories
                if (p, f) in supply.keys()) == sum(demand[p, s]
                                                   for s in stores
                                                   if (p, s) in demand.keys()))
# model
model = ConcreteModel()

# decision variables
model.transpFW = Var(FWroutes, domain=NonNegativeReals)
model.transpWS = Var(WSroutes, domain=NonNegativeReals)

# objective function
model.total_cost = Objective(expr=sum((FWcost[p, f, w] * model.transpFW[p, f, w]) for (p, f, w) in FWroutes)+sum((WScost[p, w, s] * model.transpWS[p, w, s]) for (p, w, s) in WSroutes), sense=minimize)

# constraints
model.supply_ct = ConstraintList()
for p in products:
     for f in factories:
            if (p, f) in supply:
                model.supply_ct.add( sum( model.transpFW[p, f, w] for w in warehouses if (p, f, w) in FWroutes) == supply[p, f] )

model.demand_ct = ConstraintList()
for p in products:
     for s in stores:
            if (p, s) in demand:
                model.demand_ct.add( sum( model.transpWS[p, w, s] for w in warehouses if (p, w, s) in WSroutes) == demand[p, s] )

model.equalIO = ConstraintList()
model.equalIO.add( sum(model.transpFW[p, f, w] for (p, f, w) in FWroutes) == sum(model.transpWS[p, w, s] for (p, w, s) in WSroutes) )

model.maxStorage = ConstraintList()
for w in warehouses:
    model.maxStorage.add( sum(model.transpFW[p, f, w] for (p,f) in supply if (p, f, w) in FWroutes) <= maxStorage[0])


model.FWcapacity = ConstraintList()
for (f, w) in fact_wares:
    model.FWcapacity.add( sum(model.transpFW[p, f, w] for p in products if (p, f, w) in FWroutes)  <= capacityFW[0])


model.WScapacity = ConstraintList()
for (w, s) in wares_stores:
    model.WScapacity.add( sum(model.transpWS[p, w, s] for p in products if (p, w, s) in WSroutes)  <= capacityWS[0])

#solve and display
solver = SolverFactory('glpk')
solver.solve(model)

# display
print(f"The minimum total transportation cost = ${model.total_cost():,.2f}")

def printFW(prod):
    print("\nTransported amounts of product", prod, "from factory to warehouse:\n")
    df = pd.DataFrame(0, index = warehouses, columns = factories)
    for (p, f, w) in FWroutes:
        if p == prod:
            df.loc[w, f] = model.transpFW[p, f, w].value
    print(df)

def printWS(prod):
    print("\nTransported amounts of product", prod, "from warehouse to store:\n")
    df = pd.DataFrame(0, index = stores, columns = warehouses)
    for (p, w, s) in WSroutes:
        if p == prod:
            df.loc[s, w] = model.transpWS[p, w, s].value
    print(df)

for p in range(len(products)):
    printFW(products[p])
    
for p in range(len(products)):
    printWS(products[p])


The minimum total transportation cost = $38,320.00

Transported amounts of product pA from factory to warehouse:

     fA   fB   fC  fD  fE
wA  600    0    0   0   0
wB  100    0    0   0   0
wC    0    0    0   0   0
wD    0    0    0   0   0
wE    0    0  800   0   0
wF    0  560    0   0   0
wG    0    0    0   0   0
wH    0    0    0   0   0
wI    0    0    0   0   0
wJ    0    0    0   0   0

Transported amounts of product pB from factory to warehouse:

     fA   fB   fC  fD  fE
wA    0    0    0   0   0
wB  500    0    0   0   0
wC    0    0    0   0   0
wD    0    0    0   0   0
wE    0  560  200   0   0
wF    0  440  400   0   0
wG    0    0    0   0   0
wH    0    0    0   0   0
wI    0    0    0   0   0
wJ    0    0    0   0   0

Transported amounts of product pC from factory to warehouse:

     fA   fB  fC  fD  fE
wA  400    0   0   0   0
wB    0    0   0   0   0
wC    0    0   0   0   0
wD    0    0   0   0   0
wE    0    0   0   0   0
wF    0    0   0   0   0
wG    0    0 

# **HW3.4** Assignment Problem 9.3-4 from the textbook

The coach of an age group swim team needs to assign swimmers to a 200-yard medley relay team to send to the Junior Olympics. Since most of his best swimmers are very fast in more than one stroke, it is not clear which swimmer should be assigned to each of the four strokes. The five fastest swimmers and the best times (in seconds) they have achieved in each of the strokes (for 50 yards) are

<img src = "images/swim.png" width="600">

The coach wishes to determine how to assign four swimmers to the four different strokes to minimize the sum of the corresponding best times.  

Treat this as an assignment problem and use Pyomo to find the optimal solution.  For full credit you must use an abstract approach to the solution code and display nicely formatted output.

<font color = "blue"> *** 10 points -  answer in cell below *** (don't delete this cell) </font>

In [10]:
from pyomo.environ import *
import pandas as pd

swimmers = ['Carl', 'Chris', 'David', 'Tony', 'Ken']
demand = dict(zip(swimmers, [1, 1, 1, 1, 1]))

strokes = ['Backstroke', 'Breaststroke', 'Butterfly', 'Freestyle', 'None']
supply = dict(zip(strokes, [1, 1, 1, 1, 1]))

bigM = 1000
stroke_times = [[37.7, 32.9, 33.8, 37.0, 35.4], [43.4, 33.1, 42.2, 34.7, 41.8], [33.3, 28.5, 38.9, 30.4, 33.6],[29.2, 26.4, 29.6, 28.5, 31.1], [0, 0, 0, 0, 0]]
times = {
    strokes[t]: {swimmers[s]: stroke_times[t][s]
                   for s in range(len(swimmers))}
    for t in range(len(strokes))
}
# throw an error if total supply and demand do not match
assert (sum(supply.values()) == sum(demand.values()))

model = ConcreteModel()

model.assign= Var(strokes, swimmers, domain=NonNegativeReals)

model.total_times = Objective(expr=sum(times[t][s] * model.assign[t, s]
                                      for t in strokes for s in swimmers),
                             sense=minimize)

model.supply_ct = ConstraintList()
for t in strokes:
    model.supply_ct.add(
        sum(model.assign[t, s] for s in swimmers) == supply[t])

model.demand_ct = ConstraintList()
for s in swimmers:
    model.demand_ct.add(
        sum(model.assign[t, s] for t in strokes) == demand[s])

# solve and display
solver = SolverFactory('glpk')
solver.solve(model)

# display solution
print(f"Sum of best times = {model.total_times():,.2f} seconds.\n")
assignments = {t:s for s in swimmers for t in strokes if model.assign[t, s]() == 1.0}
print("Assignments")
print("---------------------------------------\n")
for assignment in assignments:
    print(assignments[assignment],": ", assignment)
print(f"\nSorry,",assignments["None"])



Sum of best times = 126.20 seconds.

Assignments
---------------------------------------

Carl :  Freestyle
Chris :  Butterfly
David :  Backstroke
Tony :  Breaststroke
Ken :  None

Sorry, Ken
